In [ ]:
# ─── Cell 1: API 키 입력 및 설정 ───
import google.generativeai as genai

GOOGLE_API_KEY = input("Google API 키를 입력하세요: ")
if not GOOGLE_API_KEY:
    raise ValueError("API 키가 입력되지 않았습니다.")

genai.configure(api_key=GOOGLE_API_KEY)
print("✅ API 설정 완료. 다음 셀을 실행하세요.")


In [ ]:
# ─── Cell 2: 모델 고정 설정 ───
import google.generativeai as genai

# 고정된 모델 설정
MODEL_NAME = "gemini-2.5-pro-exp-03-25"

try:
    model = genai.GenerativeModel(MODEL_NAME)
    print(f"✅ 모델 설정 완료: {MODEL_NAME}")
    print("다음 셀(Cell 3)을 실행하세요.")
except Exception as e:
    print(f"❌ 모델 설정 실패: {str(e)}")
    print("⚠️ Google Cloud Console에서 API 쿼터를 확인하거나, 네트워크 상태를 점검하세요.")

In [ ]:
# ─── Cell 3: 대화형 인터랙티브 모드 ───
import time
import google.generativeai as genai

# 한국어 응답을 위한 설정
SYSTEM_PROMPT = "모든 응답을 한국어로 제공하세요. 간결하고 명확하게 답변하세요."

print("💬 프롬프트를 입력하세요. 종료하려면 'exit' 또는 'quit'를 입력하세요.\n")

while True:
    prompt = input("프롬프트: ")
    if prompt.strip().lower() in ("exit", "quit"):
        print("🛑 대화를 종료합니다.")
        break
    if not prompt.strip():
        print("⚠️ 빈 입력입니다. 프롬프트를 입력하세요.\n")
        continue

    try:
        # 프롬프트와 시스템 프롬프트 결합
        full_prompt = f"{SYSTEM_PROMPT}\n\n사용자: {prompt}"
        response = model.generate_content(full_prompt)
        print("\n✅ 응답 성공!")
        print(response.text, "\n" + "-"*50 + "\n")
    except Exception as e:
        msg = str(e)
        if "503" in msg:
            print("⚠️ 503 오류: 서버가 응답하지 않습니다. 3초 후 재시도...")
            time.sleep(3)
            try:
                response = model.generate_content(full_prompt)
                print("\n✅ 재시도 성공!")
                print(response.text, "\n" + "-"*50 + "\n")
            except:
                print("❌ 재시도 실패: 네트워크나 쿼터를 확인하세요.\n" + "-"*50 + "\n")
        elif "quota" in msg.lower():
            print("❌ 쿼터 초과: Google Cloud Console에서 쿼터를 확인하세요.\n" + "-"*50 + "\n")
        else:
            print(f"❌ 오류: {msg}\n" + "-"*50 + "\n")

    # 서버 부하 방지를 위해 잠시 대기
    time.sleep(1)